<a href="https://colab.research.google.com/github/kaanbaycan/IE-423-Forecasting-Tasks/blob/main/IE_423_Task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import numpy as np
import pandas as pd
from google.colab import files

In [6]:
def upload_dataset(from_drive = False):
  if not os.path.exists("kaggle.json"):
    if not from_drive:
      print("Upload Kaggle API Key")
      files.upload()
      print("Downloading dataset...")
    else:
      !cp /content/drive/MyDrive/kaggle.json /content/
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d hariharanpavan/bank-marketing-dataset-analysis-classification
    !unzip bank-marketing-dataset-analysis-classification.zip
  else:
    print("Dataset already exists")

In [7]:
upload_dataset()

Upload Kaggle API Key


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification
License(s): GNU Lesser General Public License 3.0
 61% 1.00M/1.63M [00:00<00:00, 1.53MB/s]
100% 1.63M/1.63M [00:00<00:00, 2.23MB/s]
Archive:  bank-marketing-dataset-analysis-classification.zip
  inflating: bank-full.csv           
  inflating: bank-names.txt          
  inflating: bank.csv                
  inflating: case study.ipynb        


In [10]:
df = pd.read_csv("bank-full.csv")

In [11]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
